In [ ]:
!pip install transformers datasets huggingface_hub torch --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments


In [ ]:
# Load dataset (IMDB is much smaller than OpenWebText)
dataset = load_dataset("imdb")

# Load GPT-2 tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# GPT-2 doesn't have a default padding token, so we set it manually
tokenizer.pad_token = tokenizer.eos_token

# Tokenization function
def tokenize_function(examples):
    model_inputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)
    model_inputs["labels"] = model_inputs["input_ids"].copy()  # Shift labels for causal language modeling
    return model_inputs

# Apply tokenization
tokenized_datasets = dataset.map(tokenize_function, batched=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
# Load GPT-2 model (for causal language modeling)
model = AutoModelForCausalLM.from_pretrained("gpt2")

# Define training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-imdb-finetuned",
    eval_strategy="epoch",  # Fixed deprecated argument
    per_device_train_batch_size=4,  # Reduce batch size to prevent memory errors
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    save_strategy="epoch",
    logging_dir="./logs",  # Log directory
    logging_steps=500,
    save_total_limit=2,  # Keep only the last 2 checkpoints
    report_to="none",  # Disable Weights & Biases logging
)


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"].shuffle(seed=42).select(range(5000)),  # Use subset to save space
    eval_dataset=tokenized_datasets["test"].shuffle(seed=42).select(range(1000))  # Use smaller validation set
)

# Start training
trainer.train()


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,2.942600,2.772708
2,2.726700,2.764638
3,2.644700,2.768109


TrainOutput(global_step=3750, training_loss=2.772529459635417, metrics={'train_runtime': 1274.7379, 'train_samples_per_second': 11.767, 'train_steps_per_second': 2.942, 'total_flos': 1959690240000000.0, 'train_loss': 2.772529459635417, 'epoch': 3.0})

In [ ]:
notebook_login()  # You will be asked to enter your Hugging Face API token


In [ ]:
# Push model to Hugging Face Hub
model.push_to_hub("Flake56/gpt2-imdb-finetuned")
tokenizer.push_to_hub("Flake56/gpt2-imdb-finetuned")


model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Flake56/gpt2-imdb-finetuned/commit/88c8d8762c1597cb68b84ccb087d11024360d4fe', commit_message='Upload tokenizer', commit_description='', oid='88c8d8762c1597cb68b84ccb087d11024360d4fe', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Flake56/gpt2-imdb-finetuned', endpoint='https://huggingface.co', repo_type='model', repo_id='Flake56/gpt2-imdb-finetuned'), pr_revision=None, pr_num=None)